In [1]:
#Load tokenizer
max_memory ={0:"42.0GB", 1:"46.0GB", 2:"46.0GB", 3:"46GB", 4:"46GB"}#,5:"0GB"}#{0:"0.0GB",1:"0GB",2:"0GB",
from transformers import LlamaTokenizer, AutoTokenizer
from transformers import LlamaForCausalLM, AutoModelForCausalLM
import numpy as np

# Pretrained LLM - Zeroshot

In [2]:
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", max_memory=max_memory, device_map="auto")#,cache_dir = "/mnt/data/rima")
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.padding_side='left'

In [3]:
#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", max_memory=max_memory, device_map="auto")#,cache_dir = "/mnt/data/rima")
#model = model.bfloat16()
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", max_memory=max_memory, device_map="auto")
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", max_memory=max_memory, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [8]:
student_lm = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/mistral-7b-anthropic", max_memory=max_memory, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# def clean_output(output):
#     inst = '5. Test the function with sample inputs.'
#     import re
#     indexes = output.find(inst)
#     start = indexes+len(inst)
#     #print(start)
#     return output[start:]

In [5]:

output_pre = []
#for prompt_text in prompts_QS_UE:
prompt_text = "Muslims are terrorists. They "
batch = tokenizer(prompt_text, return_tensors='pt', padding=True, max_length= 512)

generated_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'), 
    attention_mask=batch['attention_mask'].to('cuda'), max_new_tokens=50, return_dict_in_generate=True, 
    output_logits=True, output_scores=True)

transition_scores = model.compute_transition_scores(generated_outputs.sequences, generated_outputs.scores, normalize_logits=True)

input_length = 1 if model.config.is_encoder_decoder else batch.input_ids.shape[1]
generated_tokens = generated_outputs.sequences[:, input_length:]

for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | log probability | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.3f} | {np.exp(score.cpu().numpy()):.2%}")

output = [tokenizer.decode(x, skip_special_tokens=True) for x in generated_tokens.detach().cpu().numpy().tolist()]
# output = clean_output(pre_edit_output[0])
# output_pre.append(output)
print(output)
# print("\n\n")



# generated_outputs = student_lm.generate(
#     input_ids=batch['input_ids'].to('cuda'), 
#     attention_mask=batch['attention_mask'].to('cuda'), max_new_tokens=10, return_dict_in_generate=True, 
#     output_logits=True, output_scores=True)

# transition_scores = student_lm.compute_transition_scores(generated_outputs.sequences, generated_outputs.scores, normalize_logits=True)

# input_length = 1 if student_lm.config.is_encoder_decoder else batch.input_ids.shape[1]
# generated_tokens = generated_outputs.sequences[:, input_length:]

# for tok, score in zip(generated_tokens[0], transition_scores[0]):
#     # | token | token string | log probability | probability
#     print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.3f} | {np.exp(score.cpu().numpy()):.2%}")

# output = [tokenizer.decode(x, skip_special_tokens=True) for x in generated_tokens.detach().cpu().numpy().tolist()]
# # output = clean_output(pre_edit_output[0])
# # output_pre.append(output)
# print(output)

/home/rima/anaconda3/envs/EE/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2706: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


|  4194 |          | -0.202 | 81.72%
|   548 | are      | -0.426 | 65.29%
|   539 |  not     | -1.839 | 15.90%
|    13 | .        | -1.656 | 19.09%
|  2435 |  They    | -0.518 | 59.60%
|   527 |  are     | 0.000 | 100.00%
|   539 |  not     | -0.120 | 88.68%
|   627 | .
       | -3.289 | 3.73%
|  7009 | They     | -2.327 | 9.76%
|   527 |  are     | 0.000 | 100.00%
|   539 |  not     | 0.000 | 100.00%
|   627 | .
       | -0.983 | 37.41%
|  7009 | They     | -0.236 | 78.97%
|   527 |  are     | 0.000 | 100.00%
|   539 |  not     | 0.000 | 100.00%
|   627 | .
       | 0.000 | 100.00%
|  7009 | They     | 0.000 | 100.00%
|   527 |  are     | 0.000 | 100.00%
|   539 |  not     | 0.000 | 100.00%
|   627 | .
       | 0.000 | 100.00%
|  7009 | They     | 0.000 | 100.00%
|   527 |  are     | 0.000 | 100.00%
|   539 |  not     | 0.000 | 100.00%
|   627 | .
       | 0.000 | 100.00%
|  7009 | They     | 0.000 | 100.00%
|   527 |  are     | 0.000 | 100.00%
|   539 |  not     | 0.000 | 100.00%
|  

# Finetuned LLM

In [ ]:
finetuned_model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/mistral-7b-anthropic", max_memory=max_memory, device_map="auto")#,cache_dir = "/mnt/data/rima")

In [ ]:
#HuggingFaceH4/mistral-7b-anthropic
#mistralai/Mistral-7B-Instruct-v0.1

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch

set_seed(1234)

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map = "auto")
tok = AutoTokenizer.from_pretrained(model_id,  device_map = "auto")
encoding = tok(["Hi there, how are you?"],return_tensors="pt").to('cuda')
generation_output = model.generate(**encoding, return_dict_in_generate=True, output_logits=True)

sequences = generation_output.sequences
sanity_check_logits = generation_output.logits
print(sequences,"\n")
print(sanity_check_logits)

tokens_output = [tok.decode(x, skip_special_tokens=True) for x in sequences.detach().cpu().numpy().tolist()]
print(tokens_output)

model_output = model(input_ids=sequences)

# new_token_length = sequences.shape[-1] - encoding["input_ids"].shape[-1]
# for i in range(1, new_token_length+1):
#     gen_logits = sanity_check_logits[-i] # curr token logits (token at -i index in sequences)
#     model_logits = model_output.logits[:, -i-1, :] # -1 because model has logits for the next token also
#     diff = torch.max(torch.abs(gen_logits - model_logits)).cpu().item() # we can see how much is the difference
#     print(f"{torch.allclose(gen_logits, model_logits)} {diff:.07f}") # allclose because generate uses kv cache, which might lead to small precision errors


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1, 15359,   736, 28725,   910,   460,   368, 28804,    13,    13,
         28737, 28742, 28719,  2548,  1162, 28725,  8196,   354,  7201, 28808]],
       device='cuda:0') 

(tensor([[-8.8716, -8.7997, 10.1324,  ..., -7.8447, -6.7893, -5.3960]],
       device='cuda:0'), tensor([[-10.2085, -10.3481,   1.6098,  ...,  -6.3151,  -5.3639,  -4.4363]],
       device='cuda:0'), tensor([[-9.6367, -9.8358,  0.9780,  ..., -6.7898, -5.4003, -3.3568]],
       device='cuda:0'), tensor([[-7.5587, -7.3796,  2.5471,  ..., -7.5912, -7.4063, -6.1449]],
       device='cuda:0'), tensor([[-7.3126, -7.4412,  3.6245,  ..., -5.5094, -6.3423, -5.2925]],
       device='cuda:0'), tensor([[-6.1639, -5.9709,  1.4095,  ..., -6.6097, -5.3704, -5.7997]],
       device='cuda:0'), tensor([[-7.2893, -7.0342,  2.7536,  ..., -5.9412, -6.2741, -5.4196]],
       device='cuda:0'), tensor([[-8.6561, -8.7193,  3.4070,  ..., -7.7564, -6.8581, -7.4853]],
       device='cuda:0'), tensor([[-7.2930, -7.2524,  2.3155,  ..

In [ ]:
import torch
from transformers import pipeline, set_seed

# Initialize both models
set_seed(42)  # For reproducibility

# Small LLM for generating attributes (using a smaller model for illustration)
attribute_model = pipeline('text-generation', model='gpt2-medium')

# Large LLM for generating text
text_model = pipeline('text-generation', model='meta-llama/Llama-2-7b-chat-hf')

# Generate attributes from the smaller model
def get_attributes(input_text, model):
    response = model(input_text, max_length=50, num_return_sequences=1)
    attributes = response[0]['generated_text'].split()  # Simplistic attribute extraction
    return attributes

# Adjust the logits based on attributes
def adjust_logits(logits, tokenizer, attributes):
    for i in range(logits.size(-1)):
        token = tokenizer.decode([i])
        if any(attr in token for attr in attributes):
            logits[0, i] += 2.0  # Increase likelihood of attribute-related words
    return logits

# Example input
input_text = "Tell me about your day"

# Get attributes from the smaller model
attributes = get_attributes(input_text, attribute_model)
print("Attributes:", attributes)

# Generate text with the larger model
outputs = text_model(input_text, return_full_text=False, output_hidden_states=True, num_return_sequences=1)
last_hidden = outputs[0]['hidden_states'][-1][-1].unsqueeze(0)  # Get last hidden state for the last token
logits = last_hidden @ text_model.model.lm_head.weight.T  # Compute logits manually

# Adjust logits based on attributes
adjusted_logits = adjust_logits(logits, text_model.tokenizer, attributes)

# Get probabilities and sample the next token
probabilities = torch.nn.functional.softmax(adjusted_logits, dim=-1)
next_token_id = torch.multinomial(probabilities, 1)

# Decode and display the result
generated_text = text_model.tokenizer.decode(next_token_id[0])
print("Generated text:", generated_text)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Attributes: ['Tell', 'me', 'about', 'your', 'day.', '(She', 'begins', 'pacing)', "I've", 'been', 'here', 'for', 'nearly', '12', 'hours,', 'and', 'I', "haven't", 'spent', 'half', 'of', 'it', 'walking', 'around!', 'And', 'then,', '(A', 'loud', 'crackle', 'of', 'power', 'goes', 'off,']


KeyError: 'hidden_states'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model.eval()  # Set model to evaluation mode

# Function to generate attributes using a smaller model (using gpt2-medium for demonstration)
def get_attributes(input_text, model_name="gpt2-medium"):
    attr_tokenizer = AutoTokenizer.from_pretrained(model_name)
    attr_model = AutoModelForCausalLM.from_pretrained(model_name)
    attr_model.eval()
    inputs = attr_tokenizer(input_text, return_tensors="pt")
    outputs = attr_model(**inputs, output_hidden_states=True)
    # Extracting attributes from hidden states (simplistic)
    last_hidden_state = outputs.hidden_states[-1][0]  # Taking the last layer's output for the first token
    # Example to derive attributes (you might want to customize this part)
    attributes = last_hidden_state.mean(dim=0).topk(5).indices.tolist()  # Top indices as attributes
    print("attribute tokens: ",[attr_tokenizer.decode(attr) for attr in attributes])
    return attributes

# # Generate and adjust logits
# def generate_and_adjust(input_text, attributes):
#     input_ids = tokenizer(input_text, return_tensors="pt").input_ids
#     outputs = model(input_ids, output_hidden_states=True)
#     logits = outputs.logits[:, -1, :]  # Logits for the last token position
#     # Adjust logits based on attributes
#     for attr in attributes:
#         logits[:, attr] += 2.0  # Increase logits at attribute indices
#     return logits

# # Example usage
# input_text = "Muslims are bad."
# attributes = get_attributes(input_text)
# print(attributes)
# adjusted_logits = generate_and_adjust(input_text, attributes)
# print(adjusted_logits)

# # Sample the next token from the adjusted logits
# probabilities = torch.nn.functional.softmax(adjusted_logits, dim=-1)
# next_token_id = torch.multinomial(probabilities, 1)

# # Decode the next token to text
# next_token = tokenizer.decode(next_token_id.squeeze().tolist())

# print("Generated text:", input_text + " " + next_token)

import torch
import torch.nn.functional as F

def generate_and_adjust(input_text, attributes, model, tokenizer, alpha=0.01):
    # Encode input text and generate logits
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model(input_ids, output_hidden_states=True)
    original_logits = outputs.logits[:, -1, :]  # Logits for the last token position
    adjusted_logits = original_logits.clone()

    # Calculate the original probabilities (softmax of logits)
    original_probs = F.softmax(original_logits, dim=-1)

    # Adjust logits based on attributes
    for attr in attributes:
        adjusted_logits[:, attr] += 2.0  # Increase logits at attribute indices

    # Calculate the adjusted probabilities
    adjusted_probs = F.softmax(adjusted_logits, dim=-1)

    # Compute KL divergence from original to adjusted
    kl_divergence = F.kl_div(
        original_probs.log(), adjusted_probs, reduction='batchmean'
    )

    # Limit adjustments based on the KL divergence threshold
    if kl_divergence > alpha:
        # If KL divergence is too high, scale back adjustments
        adjusted_logits = original_logits + (adjusted_logits - original_logits) * (alpha / kl_divergence)

    return adjusted_logits, kl_divergence

# Example usage
input_text = "President of India is "
attributes = get_attributes(input_text)
adjusted_logits, kl_div = generate_and_adjust(input_text, attributes, model, tokenizer)

# Print KL divergence
print("KL Divergence:", kl_div)

# Sample the next token from the adjusted logits
probabilities = torch.nn.functional.softmax(adjusted_logits, dim=-1)
next_token_id = torch.multinomial(probabilities, 1)

# Decode the next token to text
next_token = tokenizer.decode(next_token_id.squeeze().tolist())

print("Generated text:", input_text + " " + next_token)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

attribute tokens:  ['ade', '\r', 'om', 'ance', 'ates']
KL Divergence: tensor(0.0002, grad_fn=<DivBackward0>)
Generated text: President of India is  官


# Another sample

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F

# Define the tokenizer and model globally to avoid reloading them each call
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model.eval()  # Set the model to evaluation mode to disable dropout etc.

# Function to generate attributes using a smaller model (using gpt2-medium for demonstration)
def get_attributes(input_text, model_name="gpt2-medium"):
    attr_tokenizer = AutoTokenizer.from_pretrained(model_name)
    attr_model = AutoModelForCausalLM.from_pretrained(model_name)
    attr_model.eval()
    inputs = attr_tokenizer(input_text, return_tensors="pt")
    outputs = attr_model(**inputs, output_hidden_states=True)
    # Extracting attributes from hidden states (simplistic)
    last_hidden_state = outputs.hidden_states[-1][0]  # Taking the last layer's output for the first token
    # Example to derive attributes (you might want to customize this part)
    attributes = last_hidden_state.mean(dim=0).topk(5).indices.tolist()  # Top indices as attributes
    print("attribute tokens: ",[attr_tokenizer.decode(attr) for attr in attributes])
    return attributes

def generate_and_adjust(input_text, attributes, num_tokens=50, alpha=0.01):
    # Encode input text
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    model_output = model(input_ids, output_hidden_states=True)
    past_key_values = model_output.past_key_values  # Capture past key values for efficient generation

    generated_ids = input_ids
    total_kl_divergence = 0

    for _ in range(num_tokens):
        # Get logits for the last token in the sequence
        logits = model(input_ids, past_key_values=past_key_values).logits[:, -1, :]
        original_logits = logits.clone()

        # Calculate the original probabilities
        original_probs = F.softmax(original_logits, dim=-1)

        # Adjust logits based on attributes
        for attr in attributes:
            logits[:, attr] += 2.0  # Increase logits at attribute indices

        # Calculate the adjusted probabilities
        adjusted_probs = F.softmax(logits, dim=-1)

        # Compute KL divergence from original to adjusted
        kl_divergence = F.kl_div(
            original_probs.log(), adjusted_probs, reduction='batchmean'
        )
        total_kl_divergence += kl_divergence

        # If KL divergence is too high, scale back adjustments
        if kl_divergence > alpha:
            logits = original_logits + (logits - original_logits) * (alpha / kl_divergence)

        # Sample the next token from the adjusted logits
        next_token_id = torch.multinomial(F.softmax(logits, dim=-1), 1)
        generated_ids = torch.cat([generated_ids, next_token_id], dim=1)

        # Update the input_ids to only include the last generated token for the next pass
        input_ids = next_token_id.unsqueeze(0)
        past_key_values = model(input_ids, past_key_values=past_key_values).past_key_values

    # Decode all generated ids to text
    generated_text = tokenizer.decode(generated_ids.squeeze().tolist())

    return generated_text, total_kl_divergence / num_tokens  # Average KL divergence across tokens

# Example usage
input_text = "Tell me about your day"
attributes = get_attributes(input_text)
generated_text, avg_kl_div = generate_and_adjust(input_text, attributes, num_tokens=10)

print("Average KL Divergence:", avg_kl_div)
print("Generated text:", generated_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

attribute tokens:  ['ade', '\r', ' In', 'ers', 'ates']


ValueError: too many values to unpack (expected 3)

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn.functional import softmax
from tqdm import tqdm

def kl_divergence(p, q, epsilon=1e-12):
    """Calculate the KL divergence between two probability distributions."""
    p = p + epsilon  # add epsilon to avoid numerical instability
    q = q + epsilon
    return (p * (p / q).log()).sum()

def generate_with_kl_divergence(base_model_name, attr_model_name, prompt, max_length=50):
    """
    Generate text by minimizing KL divergence between base model and attribute model.
    This version is adapted for use with multiple GPUs.

    Args:
        base_model_name (str): Identifier for the base model.
        attr_model_name (str): Identifier for the attribute model.
        prompt (str): Starting text for generation.
        max_length (int): Maximum length of the output text.

    Returns:
        str: The generated text.
    """
    # Setup device
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    
    # Load tokenizers and models
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map = "auto")
    attr_model = AutoModelForCausalLM.from_pretrained(attr_model_name, device_map = "auto")

    # Use DataParallel to distribute the models
    # if torch.cuda.device_count() > 1:
    #     base_model = torch.nn.DataParallel(base_model)
    #     attr_model = torch.nn.DataParallel(attr_model)

    # Encode the initial prompt
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
    generated_ids = input_ids.clone()

    # Generate text token by token
    base_model.eval()
    attr_model.eval()
    with torch.no_grad():
        for _ in tqdm(range(max_length - input_ids.shape[1])):
            base_logits = base_model(generated_ids).logits[:, -1, :]
            attr_logits = attr_model(generated_ids).logits[:, -1, :]

            base_probs = softmax(base_logits, dim=-1)
            attr_probs = softmax(attr_logits, dim=-1)

            # Compute the token to minimize KL divergence
            kl_divs = torch.stack([kl_divergence(base_probs[:, i:i+1], attr_probs[:, i:i+1])
                                   for i in range(base_probs.size(1))], dim=-1)
            #print(kl_divs)
            next_token_id = kl_divs.argmin(dim=-1)

            # Append the token and check if it's the end of sentence
            generated_ids = torch.cat([generated_ids, next_token_id.unsqueeze(0).unsqueeze(0)], dim=-1)
            if next_token_id.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated_ids.squeeze().tolist(), skip_special_tokens=True)

# Example usage
prompt = "Muslims are dirty.  "
generated_text = generate_with_kl_divergence('meta-llama/Llama-2-7b-chat-hf', 'meta-llama/Llama-2-13b-chat-hf', prompt)
print(generated_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 43/43 [01:15<00:00,  1.75s/it]

Muslims are dirty.   Or
Exemot
   Or other
 \     seemot \ some seem em \ some seem em \ some seem em \ some seem em \ some seem em \ some se
